**导入**

In [1]:
import sys

sys.path.append("src/")

In [2]:
import os
from collections import namedtuple
from time import time

import numpy as np
import pandas as pd
import seaborn as sns
from ortools.sat.python import cp_model
from matplotlib import pyplot as plt
from tqdm import tqdm

import config, dataset, main, utils, bins

In [3]:
%matplotlib notebook
%load_ext autoreload
%load_ext nb_black
%autoreload 2

<IPython.core.display.Javascript object>

In [4]:
plt.style.use("seaborn")
np.random.seed(config.RANDOM_SEED)

C:\Users\12605\AppData\Local\Temp\ipykernel_7484\3634308497.py:1: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


<IPython.core.display.Javascript object>

*初始维度对应平行*

**数据读取**

In [5]:
instance1_para = pd.read_csv("mydf1_para.csv")  # 货箱的初始维度：最长边与托盘高平行，次长边与托盘长平行，最短边与托盘宽平行

instance1_para.head()

,id,width,depth,height,weight,volume
0,3.034854e+11,430.0,350.0,430.0,50.248611,60200000.0
1,3.034854e+11,430.0,350.0,430.0,50.248611,60200000.0
2,3.034854e+11,430.0,350.0,430.0,50.248611,60200000.0
3,3.034854e+11,430.0,350.0,430.0,50.248611,60200000.0
4,3.034854e+11,430.0,350.0,430.0,50.248611,60200000.0


<IPython.core.display.Javascript object>

In [6]:
instance1 = pd.read_csv("mydf1.csv")  # 分布符合初始维度较好参数

instance1.head()

,id,width,depth,height,weight,volume
0,303485406111,350.0,430.0,400.0,50.248611,60200000.0
1,303485406111,350.0,430.0,400.0,50.248611,60200000.0
2,303485406111,350.0,430.0,400.0,50.248611,60200000.0
3,303485406111,350.0,430.0,400.0,50.248611,60200000.0
4,303485406111,350.0,430.0,400.0,50.248611,60200000.0


<IPython.core.display.Javascript object>

*分布符合初始维度较好参数*

In [ ]:
plt.figure()
axH = sns.histplot(
    instance1.height / instance1.width,
    bins=np.arange(0, 2, 0.05),
)
axH.set(xlabel="HEIGHT-LENGTH RATIO")
plt.show()

In [ ]:
plt.figure()
axW = sns.histplot(
    instance1.depth / instance1.width,
    bins=np.arange(0, 2, 0.05),
)
axW.set(xlabel="WIDTH-LENGTH RATIO")
plt.show()

In [ ]:
plt.figure()
axV = sns.histplot(instance1.volume / 1e6, bins=np.arange(20, 100, 2))
axV.set(xlabel="VOLUME(dm3)")
plt.show()

**初始维度对应平行：旋转扰动**

In [7]:
def swap_values(group):
    n_samples = int(
        len(group) * 0.3
    )  # calculate number of samples based on 30% threshold
    idx_samples = group.sample(n_samples).index  # select random indices from the group
    group.loc[idx_samples, ["width", "height"]] = group.loc[
        idx_samples, ["height", "width"]
    ].values  # exchange values of selected items
    return group


# group the data by `id` column and apply the `swap_values()` function
instance1_para_ry3_g = instance1_para.groupby("id").apply(swap_values)
instance1_para_ry3 = instance1_para_ry3_g.reset_index(drop=True)

# print the results
print(instance1_para_ry3)

               id  width  depth  height      weight      volume
0    7.893258e+10  510.0  320.0   590.0  141.153204  96288000.0
1    7.893258e+10  590.0  320.0   510.0  141.153204  96288000.0
2    7.893258e+10  510.0  320.0   590.0  141.153204  96288000.0
3    7.893258e+10  510.0  320.0   590.0  141.153204  96288000.0
4    7.893258e+10  510.0  320.0   590.0  141.153204  96288000.0
..            ...    ...    ...     ...         ...         ...
199  9.165267e+11  700.0  290.0   490.0    0.157856  99470000.0
200  9.165267e+11  490.0  290.0   700.0    0.157856  99470000.0
201  9.165267e+11  490.0  290.0   700.0    0.157856  99470000.0
202  9.165267e+11  490.0  290.0   700.0    0.157856  99470000.0
203  9.165267e+11  490.0  290.0   700.0    0.157856  99470000.0

[204 rows x 6 columns]


<IPython.core.display.Javascript object>

**初始维度对应平行：装箱运算**

In [8]:
bin_pool_para_ry3 = main.main(
    instance1_para_ry3,
    procedure="mr",
    superitems_horizontal=False,
    superitems_max_vstacked=2,
    superitems_horizontal_type="two-width",
)
bin_pool_para_ry3.get_original_layer_pool().to_dataframe()


2023-05-26 14:36:23.652 | INFO     | main:main:174 - MR procedure starting
2023-05-26 14:36:23.653 | INFO     | main:main:184 - MR iteration 1/1
2023-05-26 14:36:23.680 | DEBUG    | superitems:_gen_single_items_superitems:674 - Generated 204 superitems with a single item
2023-05-26 14:36:23.681 | INFO     | superitems:gen_superitems:629 - Generating vertical superitems with maximum stacking of 2
2023-05-26 14:36:23.689 | DEBUG    | superitems:_gen_superitems_vertical:833 - Generated 102 wide vertical superitems
2023-05-26 14:36:23.691 | DEBUG    | superitems:_gen_superitems_vertical:835 - Generated 0 deep vertical superitems
2023-05-26 14:36:23.692 | INFO     | superitems:gen_superitems:633 - Generated 306 superitems
2023-05-26 14:36:23.696 | INFO     | superitems:gen_superitems:635 - Remaining superitems after filtering by pallet dimensions: 306
2023-05-26 14:36:23.701 | INFO     | main:maxrects_warm_start:44 - MR computing layers
2023-05-26 14:36:23.702 | INFO     | main:maxrects_war

2023-05-26 14:36:26.447 | DEBUG    | maxrects:maxrects_single_layer_online:138 - MR-SL-Online starting
2023-05-26 14:36:26.448 | DEBUG    | maxrects:maxrects_single_layer_online:147 - MR-SL-Online 5 non-zero duals to place
2023-05-26 14:36:26.452 | DEBUG    | maxrects:maxrects_single_layer_online:196 - MR-SL-Online generated a new layer with 5 superitems (of which 5 with non-zero dual) and 0.5634171907756813 3D density
2023-05-26 14:36:26.454 | DEBUG    | maxrects:maxrects_multiple_layers:17 - MR-ML-Offline starting
2023-05-26 14:36:26.456 | DEBUG    | maxrects:maxrects_multiple_layers:18 - MR-ML-Offline not_used as warm_start
2023-05-26 14:36:26.458 | DEBUG    | maxrects:maxrects_multiple_layers:19 - MR-ML-Offline 62 superitems to place
2023-05-26 14:36:26.594 | DEBUG    | maxrects:maxrects_multiple_layers:74 - MR-ML-Offline generated 12 layers with 3D densities [0.7966457023060797, 0.7957296125997568, 0.7808596353940237, 0.7707472297094938, 0.7484298121764429, 0.7197535618020793, 0.7

,item,x,y,z,width,depth,height,layer
0,185,0,0,0,700,290,490,0
1,195,0,0,490,700,290,490,0
2,193,0,290,0,700,290,490,0
3,196,0,290,490,700,290,490,0
4,191,0,580,0,700,290,490,0
...,...,...,...,...,...,...,...,...
199,80,670,0,0,530,290,700,29
200,125,0,0,0,670,390,670,30
201,135,0,390,0,670,390,670,30
202,126,0,0,0,670,390,670,31


<IPython.core.display.Javascript object>

In [9]:
num_compact_bins_para_ry3 = bin_pool_para_ry3.compact_bins_num
print("Number of bins used:", num_compact_bins_para_ry3)

Number of bins used: 16


<IPython.core.display.Javascript object>

In [ ]:
bin_para_ry3_df = bin_pool_para_ry3.get_original_layer_pool().to_dataframe()
bin_para_ry3_df.to_csv('bin_para_ry3_layerResult.csv', index=False)


In [10]:
bin_para_ry3_bindf = bin_pool_para_ry3.get_original_bin_pool().to_dataframe()
bin_para_ry3_bindf.to_csv("bin_para_ry3_binResult.csv", index=False)

<IPython.core.display.Javascript object>

In [11]:
bin_para_ry3_bindesdf = bin_pool_para_ry3.get_original_bin_pool().describe()
bin_para_ry3_bindesdf.to_csv("bin_para_ry3_binResultDes.csv", index=False)

<IPython.core.display.Javascript object>

**旋转扰动**

*30%Y*

In [ ]:
def swap_values(group):
    n_samples = int(
        len(group) * 0.3
    )  # calculate number of samples based on 30% threshold
    idx_samples = group.sample(n_samples).index  # select random indices from the group
    group.loc[idx_samples, ["width", "height"]] = group.loc[
        idx_samples, ["height", "width"]
    ].values  # exchange values of selected items
    return group


# group the data by `id` column and apply the `swap_values()` function
instance1_ry3_g = instance1.groupby("id").apply(swap_values)
instance1_ry3 = instance1_ry3_g.reset_index(drop=True)

# print the results
print(instance1_ry3)

In [ ]:
plt.figure()
axW3 = sns.histplot(
    instance1_ry3.depth / instance1_ry3.width,
    bins=np.arange(0, 2, 0.05),
)
axW3.set(xlabel="WIDTH-LENGTH RATIO")
plt.show()

In [ ]:
plt.figure()
axH3 = sns.histplot(
    instance1_ry3.height / instance1_ry3.width,
    bins=np.arange(0, 2, 0.05),
)
axH3.set(xlabel="HEIGHT-LENGTH RATIO")
plt.show()

*50%Y*

In [ ]:
perc = 0.5


def swap_values(group):
    n_samples = int(
        len(group) * perc
    )  # calculate number of samples based on 30% threshold
    idx_samples = group.sample(n_samples).index  # select random indices from the group
    group.loc[idx_samples, ["width", "height"]] = group.loc[
        idx_samples, ["height", "width"]
    ].values  # exchange values of selected items
    return group


# group the data by `id` column and apply the `swap_values()` function
instance1_ry5_g = instance1.groupby("id").apply(swap_values)
instance1_ry5 = instance1_ry5_g.reset_index(drop=True)

# print the results
print(instance1_ry5)

**装箱运算**

*0%Y*

In [ ]:
bin_pool_npt = main.main(
    instance1,
    procedure="mr",
    superitems_horizontal=False,
    superitems_max_vstacked=2,
    superitems_horizontal_type="two-width",
)
bin_pool_npt.get_original_layer_pool().to_dataframe()
# really banned horizontal

In [ ]:
num_compact_bins_npt = bin_pool_npt.compact_bins_num
print("Number of bins used:", num_compact_bins_npt)

In [ ]:
bin_pool_npt.get_original_bin_pool().describe()

In [ ]:
bin_pool_npt.get_original_layer_pool().describe()

In [ ]:
bin_pool_npt.get_original_bin_pool().plot()

*30%Y*

In [ ]:
bin_pool_ry3 = main.main(
    instance1_ry3,
    procedure="mr",
    superitems_horizontal=False,
    superitems_max_vstacked=2,
    superitems_horizontal_type="two-width",
)
bin_pool_ry3.get_original_layer_pool().to_dataframe()


In [ ]:
num_compact_bins_ry3 = bin_pool_ry3.compact_bins_num
print("Number of bins used:", num_compact_bins_ry3)

In [ ]:
bin_pool_ry3.get_original_bin_pool().describe()

In [ ]:
bin_pool_ry3.get_original_layer_pool().describe()

*50%Y*

In [ ]:
bin_pool_ry5 = main.main(
    instance1_ry5,
    procedure="mr",
    superitems_horizontal=False,
    superitems_max_vstacked=2,
    superitems_horizontal_type="two-width",
)
bin_pool_ry5.get_original_layer_pool().to_dataframe()


In [ ]:
num_compact_bins_ry5 = bin_pool_ry5.compact_bins_num
print("Number of bins used:", num_compact_bins_ry5)

In [ ]:
bin_pool_ry5.get_original_bin_pool().describe()

**记录结果**

In [ ]:
bin_pool_npt.plot()

In [ ]:
bin_pool_ry3.plot()

In [ ]:
bin_ry3_df = bin_pool_ry3.get_original_layer_pool().to_dataframe()
bin_ry3_df.to_csv('bin_ry3_layerResult.csv', index=False)


In [ ]:
bin_ry3_bindf = bin_pool_ry3.get_original_bin_pool().to_dataframe()
bin_ry3_bindf.to_csv("bin_ry3_binResult.csv", index=False)

In [ ]:
bin_ry3_bindesdf = bin_pool_ry3.get_original_bin_pool().describe()
bin_ry3_bindesdf.to_csv("bin_ry3_binResultDes.csv", index=False)

In [ ]:
bin_npt_df = bin_pool_npt.get_original_layer_pool().to_dataframe()
bin_npt_df.to_csv('bin_npt_layerResult.csv', index=False)


In [ ]:
bin_npt_bindf = bin_pool_npt.get_original_bin_pool().to_dataframe()
bin_npt_bindf.to_csv("bin_npt_binResult.csv", index=False)

In [ ]:
bin_npt_bindesdf = bin_pool_npt.get_original_bin_pool().describe()
bin_npt_bindesdf.to_csv("bin_npt_binResultDes.csv", index=False)

In [ ]:
bin_ry5_df = bin_pool_ry5.get_original_layer_pool().to_dataframe()
bin_ry5_df.to_csv('bin_ry5_layerResult.csv', index=False)


In [ ]:
bin_ry5_bindf = bin_pool_ry5.get_original_bin_pool().to_dataframe()
bin_ry5_bindf.to_csv("bin_ry5_binResult.csv", index=False)

In [ ]:
bin_ry5_bindesdf = bin_pool_ry5.get_original_bin_pool().describe()
bin_ry5_bindesdf.to_csv("bin_ry5_binResultDes.csv", index=False)